## Import tools

In [149]:
import numpy as np
import pandas as pd

## Get the data

In [150]:
col_names = ['id', 'col1', 'col2', 'col3','col4','col5','col6','result', ]
data = pd.read_csv("data.csv", skiprows=1, header=None, names=col_names)
data = data.drop(columns=['id','col6'])
data.head(10)

,col1,col2,col3,col4,col5,result
0,15.775386,13.586879,8.725890,63.813564,19.718734,2
1,3.649369,1.904802,0.000000,197.210213,19.045471,3
2,17.264362,13.700638,8.733333,143.636181,17.621141,2
3,12.705183,9.485389,1.747626,82.636672,12.209535,3
4,7.573969,1.463106,-6.541057,19.726047,9.215187,1
5,11.749524,4.177473,-2.937162,146.911825,2.796695,2
6,9.847899,8.203223,-0.000000,127.461258,14.401656,3
7,18.143625,14.362603,8.733333,14.768375,19.007228,2
8,19.790224,14.473328,8.733333,24.101633,6.041166,1
9,7.146955,2.364196,-7.870026,196.319242,18.483607,2


## Node class

In [151]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

## Tree class

In [152]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

## Train-Test split

In [153]:
data = np.array(data)
np.random.shuffle(data)
data_train = data[0:1200]
data_test = data[1200:data.shape[0]]
X_train = data_train[:, :-1]
Y_train = data_train[:, -1]
X_test = data_test[:, :-1]
Y_test = data_test[:, -1]

In [154]:
Y_train = Y_train.reshape(-1,1)

## Fit the model

In [156]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_4 <= 11.307034 ? 0.17823740064177995
 left:X_2 <= 4.695439 ? 0.2628166000007015
  left:X_2 <= -4.734839 ? 0.234801467526637
    left:X_3 <= 128.467831 ? 0.4330398630967478
        left:2.0
        right:1.0
    right:X_3 <= 121.792291 ? 0.13954493920637134
        left:2.0
        right:2.0
  right:X_3 <= 134.230207 ? 0.2346852160536479
    left:X_3 <= 41.651678 ? 0.70449523555977
        left:1.0
        right:2.0
    right:X_4 <= 10.489267 ? 0.25459591075618676
        left:1.0
        right:2.0
 right:X_2 <= 3.572722 ? 0.17938057150829034
  left:X_2 <= -4.333333 ? 0.16497367819473063
    left:X_3 <= 41.704475 ? 0.13909632657476478
        left:2.0
        right:3.0
    right:X_3 <= 102.847612 ? 0.2001759882676124
        left:3.0
        right:3.0
  right:X_4 <= 19.494703 ? 0.2473616468576807
    left:X_3 <= 122.73647 ? 0.2810554293989119
        left:3.0
        right:2.0
    right:X_1 <= 10.018742 ? 0.3034315795604341
        left:2.0
        right:1.0


## Test the model

In [159]:
Y_pred = classifier.predict(X_test) 

In [167]:
def acurracy(Y1,Y2):
    total = 0
    for i in range(len(Y1)):
        if(Y1[i] == Y2[i]):
            total += 1
    return total/len(Y1)

def precision(Y1,Y2):
    vp = 0
    tp = 0
    for i in range(len(Y1)):
        if (Y1[i] == 1 or Y1[i] == 2):
            tp += 1
            if(Y2[i] == 1 or Y1[i] == 2):
                vp += 1
    return vp/tp

def recall(Y1, Y2):
    vp = 0
    t = 0
    for i in range(len(Y1)):
        if(Y2[i] == 1 or Y2[i] == 2):
            t += 1
            if(Y1[i] == 1 or Y1[i] == 2):
                vp += 1
    return vp/t   
        
def f_measure(Y1,Y2):
    _precision = precision(Y1,Y2)
    _recall = recall(Y1,Y2)
    return 2*_precision*_recall/(_precision+_recall)

In [161]:
acurracy(Y_pred,Y_test)

0.7625418060200669

In [164]:
precision(Y_pred,Y_test)

0.9585492227979274

In [166]:
recall(Y_pred,Y_test)

0.8301886792452831

In [168]:
f_measure(Y_pred,Y_test)

0.8897633491829261